## Topic 1: Data Collection, Preparation, and Storage

In [1]:
## packages needed for all functions
import os
from datetime import datetime
import pymongo
import json
import requests
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

#
#
# Section 1: Data Collection - Yelp JSON Data to MongoDB
#
#

# Function: Reads data from a JSON file and stores it in MongoDB
def yelp_json_to_mongoDB(path, file_name):
    documents = []
    
    ## connect to mongodb
    py_client = pymongo.MongoClient("mongodb://localhost:27017/")
    py_db = py_client["yelp"]
    py_col = py_db[file_name]

    ## drop collection to start fresh
    py_col.drop()

    with open(path+file_name+'.json', 'r') as file:
        ## read each json line from json file as a dict. Save all dicts to a list.
        for line in file:
            doc = json.loads(line)
            ## insert one document into the mongodb collection
            py_col.insert_one(doc)
            #documents.append(doc)
    
    ## count the no. of documents in the collection
    #print("Collection: "+file_name+" | Count: " + str(py_col.count_documents({})))
    
#
#
# Section 2: Data Collection - Zomato API Data to MongoDB
#
#

# Function: Reads data from API and stores in MongoDB
def zomato_api_to_mongoDB():
    
    ## connect to mongodb
    py_client = pymongo.MongoClient("mongodb://localhost:27017/")
    py_db = py_client["zomato"]
    py_col = py_db["restaurant"]

    ## drop collection to start fresh
    py_col.drop()
    
    ## list of coordinates of top 12 cities with the most businesses in yelp. Only restaurants
    ## from cities will be pulled from the Zomato API.
    lat = ['36.114647',
            '43.65107',
            '33.448376',
            '35.227085',
            '33.501324',
            '51.049999',
            '40.440624',
            '45.508888',
            '33.424564',
            '33.427204',
            '33.307575',
            '41.505493'
            ]
    long = ['-115.172813',
            '-79.347015',
            '-112.074036',
            '-80.843124',
            '-111.925278',
            '-114.066666',
            '-79.995888',
            '-73.561668',
            '-111.833267',
            '-111.939896',
            '-111.84494',
            '-81.68129'
            ]
    headers = {
        'Accept': 'application/json',
        'user-key': 'dde4241f1d6e955145ea0dfe0c673e49',
    }
    
    ## iterate through each city's coordinates
    for i in range(len(lat)):
        #print('City: {} | Lat: {} | Long: {}'.format(i+1, lat[i], long[i]))
        cnt = 0
        
        ## 101, since only a maximum of 100 entries is pulled in total for each city, 
        ## and a maximum of 20 entries for each API request.
        while cnt < 101:
            req = (requests.get('https://developers.zomato.com/api/v2.1/search?start={}&count=20&lat={}&lon={}' \
                                .format(cnt, lat[i], long[i]), headers=headers)).json()
            
            for rest in req["restaurants"]:
                doc = rest["restaurant"]
                doc.pop("R", None)
                doc.pop("all_reviews", None)
                doc.pop("events_url", None)
                doc.pop("featured_image", None)
                doc.pop("menu_url", None)
                doc.pop("photos", None)
                doc.pop("photos_url", None)
                doc.pop("thumb", None)
                doc.pop("apikey", None)
                doc.pop("url", None)
                py_col.insert_one(doc)
            cnt = cnt + 20

#
#
# Section 3: Data Preparation - Create Tables in Postgres
#
#

# Function: Creates Postgres tables
def create_tables_postgres():
    ## open db connection
    pg_conn = psycopg2.connect(database='postgres',
                               host='localhost',
                               port = '5432',
                               user='postgres',
                               password='postgres')
    pg_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) ## set auto commit on
    pg_cursor = pg_conn.cursor() ## open db cursor

    ## recreate database, if already exists
    pg_cursor.execute('DROP DATABASE IF EXISTS yelp;')
    pg_cursor.execute('CREATE DATABASE yelp;')

    ## creating UNLOGGED tables to increase the write performance
    ## e.g.: For 192,609 records, it took 45 minutes without 'UNCLOGGED', and 3 minutes with 'UNCLOGGED'.
    q_business = '''
    CREATE UNLOGGED TABLE IF NOT EXISTS business (
    business_id CHAR (22) PRIMARY KEY,
    name VARCHAR (100),
    address VARCHAR (520),
    city VARCHAR (60),
    state CHAR (3),
    postal_code VARCHAR (10),
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    stars FLOAT8,
    review_count INTEGER,
    is_open INTEGER,
    attributes VARCHAR (2000),
    categories VARCHAR (2000),
    hours VARCHAR (520)
    );
    '''

    q_users = '''
    CREATE UNLOGGED TABLE IF NOT EXISTS users (
    users_id CHAR (22) PRIMARY KEY,
    name VARCHAR (50),
    review_count INTEGER,
    yelping_since DATE,
    useful INTEGER,
    funny INTEGER,
    cool INTEGER,
    elite VARCHAR (100),
    friends VARCHAR (1000000),
    fans INTEGER,
    average_stars FLOAT8,
    compliment_hot INTEGER,
    compliment_more INTEGER,
    compliment_profile INTEGER,
    compliment_cute INTEGER,
    compliment_list INTEGER,
    compliment_note INTEGER,
    compliment_plain INTEGER,
    compliment_cool INTEGER,
    compliment_funny INTEGER,
    compliment_writer INTEGER,
    compliment_photos INTEGER
    );
    '''
    
    q_tip = '''
    CREATE UNLOGGED TABLE IF NOT EXISTS tip (
    business_id CHAR (22) REFERENCES business (business_id),
    users_id CHAR (22) REFERENCES users (users_id),
    text VARCHAR (5000),
    date CHAR (22),
    compliment_count INTEGER
    );
    '''

    q_checkin = '''
    CREATE UNLOGGED TABLE IF NOT EXISTS checkin (
    business_id CHAR (22) REFERENCES business (business_id),
    date VARCHAR (5000000)
    );
    '''

    q_review = '''
    CREATE UNLOGGED TABLE IF NOT EXISTS review (
    review_id CHAR (22) PRIMARY KEY,
    users_id CHAR (22) REFERENCES users (users_id),
    business_id CHAR (22) REFERENCES business (business_id),
    stars FLOAT8,
    useful INTEGER,
    funny INTEGER,
    cool INTEGER,
    text VARCHAR (20000),
    date CHAR (22)
    );
    '''
    
    q_restaurant = '''
    CREATE UNLOGGED TABLE IF NOT EXISTS restaurant (
    restaurant_id VARCHAR (15) PRIMARY KEY,
    name VARCHAR(100),
    city VARCHAR (50),
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    zipcode VARCHAR (50),
    cuisines VARCHAR (100),
    average_cost_for_two INTEGER,
    aggregate_rating FLOAT8,
    rating_text VARCHAR (20),
    votes INTEGER,
    all_reviews_count INTEGER,
    photo_count INTEGER,
    has_online_delivery INTEGER
    );
    '''
    
    try:
        ## delete tables if already exist
        pg_cursor.execute('DROP TABLE IF EXISTS business CASCADE;')
        pg_cursor.execute('DROP TABLE IF EXISTS users CASCADE;')
        pg_cursor.execute('DROP TABLE IF EXISTS tip CASCADE;')
        pg_cursor.execute('DROP TABLE IF EXISTS checkin CASCADE;')
        pg_cursor.execute('DROP TABLE IF EXISTS review CASCADE;')
        pg_cursor.execute('DROP TABLE IF EXISTS restaurant CASCADE;')
        
        ## create tables in postgres
        pg_cursor.execute(q_business)
        pg_cursor.execute(q_users)
        pg_cursor.execute(q_tip)
        pg_cursor.execute(q_checkin)
        pg_cursor.execute(q_review)
        pg_cursor.execute(q_restaurant)
    
    except (Exception, psycopg2.Error) as dbError:
        print ("Error while connecting to PostgreSQL", dbError)
    finally:
        if(pg_conn): pg_conn.close()

#
#
# Section 4: Data Preparation - Read from MongoDB, Clean, Write to Postgres
#
#

# Function: Escapes single and double quotes since attributes from json.dumps() are enclosed in double quotes. Double
# quotes cause problems in SQL INSERT statements.
def esc(s):
    if s == 'null':
        return '\'\''
    # E tells postgres to use backslash before apostrophe to escape the apostrophe
    return 'E\''+s.replace('\\','\\\\').replace('"', '\\\"').replace("'", "\\\'")+'\'' if isinstance(s, str) else '\'\''

# Function: Reads data from MongoDB and stores in Postgres
def write_tables_postgres():
    
    ## Connect to MongoDB
    m_client = pymongo.MongoClient("mongodb://localhost:27017/")
    m_col = ""
    
    ## Connect to Postgres
    pg_conn = psycopg2.connect(database='postgres',
                               host='localhost',
                               port = '5432',
                               user='postgres',
                               password='postgres')
    pg_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    pg_cursor = pg_conn.cursor()
    
    for col in ['business', 'checkin', 'tip', 'user', 'review']:
        m_col = m_client["yelp"][col] ## connect to the yelp db MongoDB
        
        if col == 'business':
            print("Info: Writing to business table...")
            for doc in m_col.find():
                query = "INSERT INTO business VALUES ("+"{},"*13+"{})"
                query = query.format(esc(doc['business_id']),
                            esc(doc['name']),
                            esc(doc['address']),
                            esc(doc['city']),
                            esc(doc['state']),
                            esc(doc['postal_code']),
                            doc['latitude'],
                            doc['longitude'],
                            doc['stars'],
                            doc['review_count'],
                            doc['is_open'],
                            esc(json.dumps(doc['attributes'])),
                            esc(doc['categories']),
                            esc(json.dumps(doc['hours'])))
                pg_cursor.execute(query)
            print("Info: Writing to business table... Completed")
            
        elif col == 'user':
            print("Info: Writing to users table...")
            for doc in m_col.find():
                query = "INSERT INTO users VALUES ("+"{},"*21+"{})"
                query = query.format(esc(doc['user_id']),
                            esc(doc['name']),
                            doc['review_count'],
                            'TO_TIMESTAMP(\''+doc['yelping_since']+'\',\'YYYY-MM-DD HH24:MI:SS\')',
                            doc['useful'],
                            doc['funny'],
                            doc['cool'],
                            esc(doc['elite']),
                            esc(doc['friends']),
                            doc['fans'],
                            doc['average_stars'],
                            doc['compliment_hot'],
                            doc['compliment_more'],
                            doc['compliment_profile'],
                            doc['compliment_cute'],
                            doc['compliment_list'],
                            doc['compliment_note'],
                            doc['compliment_plain'],
                            doc['compliment_cool'],
                            doc['compliment_funny'],
                            doc['compliment_writer'],
                            doc['compliment_photos'])
                pg_cursor.execute(query)
            print("Info: Writing to users table... Completed")
            
        elif col == 'review':
            print("Info: Writing to review table...")
            for doc in m_col.find():
                query = "INSERT INTO review VALUES ("+"{},"*8+"{})"
                query = query.format(esc(doc['review_id']),
                            esc(doc['user_id']),
                            esc(doc['business_id']),
                            doc['stars'],
                            doc['useful'],
                            doc['funny'],
                            doc['cool'],
                            esc(doc['text']),
                            'TO_TIMESTAMP(\''+doc['date']+'\',\'YYYY-MM-DD HH24:MI:SS\')')
                pg_cursor.execute(query)
            print("Info: Writing to review table... Completed")
            
        elif col == 'tip':
            print("Info: Writing to tip table...")
            for doc in m_col.find():
                query = "INSERT INTO tip VALUES ("+"{},"*4+"{})"
                query = query.format(esc(doc['business_id']),
                            esc(doc['user_id']),
                            esc(doc['text']),
                            'TO_TIMESTAMP(\''+doc['date']+'\',\'YYYY-MM-DD HH24:MI:SS\')',
                            doc['compliment_count'])
                pg_cursor.execute(query)
            print("Info: Writing to tip table... Completed")
            
        elif col == 'checkin':
            print("Info: Writing to checkin table...")
            for doc in m_col.find():
                query = "INSERT INTO checkin VALUES ({},{})"
                query = query.format(esc(doc['business_id']),
                            esc(doc['date']))
                pg_cursor.execute(query)
            print("Info: Writing to checkin table... Completed")
    
    for col in ['restaurant']:
        m_col = m_client["zomato"][col] ## connect to the zomato db in MongoDB
        
        if col == 'restaurant':               
            print("Info: Writing to restaurant table...")
            for doc in m_col.find():
                query = "INSERT INTO restaurant VALUES ("+"{},"*13+"{})"
                query = query.format(esc(doc['id']),
                            esc(doc['name']),
                            esc(doc['location']['city']),
                            doc['location']['latitude'],
                            doc['location']['longitude'],
                            esc(doc['location']['zipcode']),
                            esc(doc['cuisines']),
                            doc['average_cost_for_two'],
                            doc['user_rating']['aggregate_rating'],
                            esc(doc['user_rating']['rating_text']),
                            doc['user_rating']['votes'],
                            doc['all_reviews_count'],
                            doc['photo_count'],
                            doc['has_online_delivery'])
                pg_cursor.execute(query)
            print("Info: Writing to restaurant table... Completed")


def run_extraction_storage():
    path = os.getcwd()+'/yelp_dataset/'
    file_names = ['business', 'checkin', 'tip', 'tip', 'user', 'review']

    print("yelp_json_to_mongoDB() - Start -> " + str(datetime.now().time()))
    for file in file_names:
        yelp_json_to_mongoDB(path, file)
    print("yelp_json_to_mongoDB() - Finish -> " + str(datetime.now().time()))
    
    print("zomato_api_to_mongoDB() - Start -> " + str(datetime.now().time()))
    zomato_api_to_mongoDB()
    print("zomato_api_to_mongoDB() - Finish -> " + str(datetime.now().time()))
    
    print("create_tables_postgres() - Start -> " + str(datetime.now().time()))
    create_tables_postgres()
    print("create_tables_postgres() - Finish -> " + str(datetime.now().time()))
    
    print("write_tables_postgres() - Start -> " + str(datetime.now().time()))
    write_tables_postgres()
    print("write_tables_postgres() - Finish -> " + str(datetime.now().time()))

## Topic 2: Data Analysis - Analysing Ratings of Yelp and Zomato

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

mergedf1 = pd.DataFrame()

def dataread():
    ## Connect to Postgres
    pg_conn = psycopg2.connect(database='postgres',
                               host='localhost',
                               port = '5432',
                               user='postgres',
                               password='Shreyshah9')
    pg_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    pg_cursor = pg_conn.cursor()
    
    #checkin, restaurant
    query = "select r.restaurant_id, b.business_id, r.name, r.city, r.aggregate_rating, b.stars, r.votes, b.review_count from restaurant r, business b where r.name = b.name and ROUND(r.latitude::numeric, 3) = ROUND(b.latitude::numeric, 3) and ROUND(r.longitude::numeric, 3) = ROUND(b.longitude::numeric, 3)"
    #print(query)
    mergedf = pd.read_sql_query(query, pg_conn)
    #display(mergedf)
    mergedf1['zom_id'] = mergedf['restaurant_id']
    mergedf1['yelp_id'] = mergedf['business_id']
    mergedf1['name'] = mergedf['name']
    mergedf1['city'] = mergedf['city']
    mergedf1['zom_ratings'] = mergedf['aggregate_rating']
    mergedf1['yelp_ratings'] = mergedf['stars']
    mergedf1['zom_votes'] = mergedf['votes']
    mergedf1['yelp_votes'] = mergedf['review_count']
    pg_conn.close()
    
## Function: 1st set of Visualizations
def run_visualization_1():
    dataread()
    
    # Plot 1
    plt.plot(mergedf1.zom_ratings, color='coral')
    plt.plot(mergedf1.yelp_ratings, color='grey')
    plt.xlabel('Restaurants')
    plt.ylabel('Ratings out of 5')
    plt.title('Yelp vs Zomato ratings comparison')
    rp = mpatches.Patch(color='coral', label='Zomato Ratings')
    gp = mpatches.Patch(color='grey', label='Yelp Ratings')
    plt.legend(handles=[rp, gp])
    plt.savefig('plot_before_adj.png', transparent=True, dpi=2000)
    plt.show()
    
    # Plot 2
    plt.plot(mergedf1.zom_votes, color='coral')
    plt.plot(mergedf1.yelp_votes, color='grey')
    plt.xlabel('Restaurant ids')
    plt.ylabel('Total number of votes')
    plt.title('Yelp vs Zomato votes comparison')
    rp = mpatches.Patch(color='coral', label='Zomato votes')
    gp = mpatches.Patch(color='grey', label='Yelp votes')
    plt.legend(handles=[rp, gp])
    plt.savefig('plot_votes.png', transparent=True, dpi=2000)
    plt.show()


    #Plot 3: Scatterplot (Here it is observed yelp has only absolute values with intervals of 0.5 and zomato has interval of 0.1)
    plt.scatter( mergedf1.index, mergedf1.zom_ratings, color='coral')
    plt.scatter( mergedf1.index, mergedf1.yelp_ratings, color='grey')
    plt.savefig('scatter_before_adj.png', transparent=True, dpi=2000)
    plt.show()
    
    #Creating new dataframe to perform further operations on duplicate database to prevent losing actual data
    C = pd.DataFrame()
    
    C['zom_ratings'] = mergedf1['zom_ratings']
    C['yelp_ratings'] = mergedf1['yelp_ratings']
    C['zom_votes'] = mergedf1['zom_votes']
    C['yelp_votes'] = mergedf1['yelp_votes']
    
    #Converting zomato ratings to absolute ratings as well like yelp for unbiased visualization
    C.loc[C['zom_ratings'] > 4.75, 'zom_ratings'] = 5
    C.loc[(C['zom_ratings'] > 4.25) & (C['zom_ratings'] <= 4.75), 'zom_ratings'] = 4.50
    C.loc[(C['zom_ratings'] > 3.75) & (C['zom_ratings'] <= 4.25), 'zom_ratings'] = 4.00
    C.loc[(C['zom_ratings'] > 3.25) & (C['zom_ratings'] <= 3.75), 'zom_ratings'] = 3.50
    C.loc[(C['zom_ratings'] > 2.75) & (C['zom_ratings'] <= 3.25), 'zom_ratings'] = 3.00
    C.loc[(C['zom_ratings'] > 2.25) & (C['zom_ratings'] <= 2.75), 'zom_ratings'] = 2.50
    C.loc[(C['zom_ratings'] > 1.75) & (C['zom_ratings'] <= 2.25), 'zom_ratings'] = 2.00
    
    yelp_mean = (C.yelp_ratings).mean()
    zom_mean = (C.zom_ratings).mean()
    
    #Plot 4: Scatterplot after converting zomato ratings to absolute values
    list_length = len(C.index)
    plt.scatter( C.index, C.zom_ratings, color='coral')
    plt.plot(C.index, [zom_mean] * list_length  , label='Mean', linestyle='--', color = "coral")
    plt.scatter( C.index, C.yelp_ratings, marker='x', color='grey')
    plt.plot(C.index, [yelp_mean] * list_length  , label='Mean', linestyle='--', color = "grey")
    plt.title('Yelp vs Zomato ratings comparison')
    rp = mpatches.Patch(color='coral', label='Zomato Ratings')
    gp = mpatches.Patch(color='grey', label='Yelp Ratings')
    plt.legend(handles=[rp, gp])
    plt.xlabel('Restaurant ids')
    plt.ylabel('Ratings out of 5')
    plt.savefig('scatter_after_adj.png', dpi=2000)
    
    #Plot 5After conversion to zomato absolute values
    plt.plot(C.zom_ratings, color='coral')
    plt.plot(C.yelp_ratings, color='grey')
    plt.xlabel('Restaurants')
    plt.ylabel('Ratings out of 5')
    plt.title('Yelp vs Zomato ratings comparison')
    rp = mpatches.Patch(color='coral', label='Zomato Ratings')
    gp = mpatches.Patch(color='grey', label='Yelp Ratings')
    plt.legend(handles=[rp, gp])
    plt.savefig('line_after_adj.png', transparent=True, dpi=2000)
    plt.show()

## Topic 3: Data Analysis - Analysing Rating Differences and how Cuisines affect them

In [2]:
import pandas as pd
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import matplotlib.pyplot as plt
import seaborn as sb
import folium

## Function: Retreives 391 restaurants from both the Yelp and Zomato datasets
def get_data(s_query='XXX'):
    ## Connect to Postgres
    pg_conn = psycopg2.connect(database='postgres',
                               host='localhost',
                               port = '5432',
                               user='postgres',
                               password='postgres')
    pg_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    pg_cursor = pg_conn.cursor()

    query = '''
    select b.name, r.restaurant_id as zom_id, b.business_id as yelp_id, 
    r.aggregate_rating as zom_rating, b.stars as yelp_rating,
    b.latitude as lat, b.longitude as long,
    r.votes as zom_rtg_count, b.review_count as yelp_rtg_count
    from restaurant r, business b
    where r.name = b.name
    and ROUND(r.latitude::numeric, 3) = ROUND(b.latitude::numeric, 3)
    and ROUND(r.longitude::numeric, 3) = ROUND(b.longitude::numeric, 3)
    '''
    
    if s_query != 'XXX':
        query = s_query

    df = pd.read_sql_query(query, pg_conn)

    pg_conn.close()
    return df

## Function: Bins the rating differences between the sites
def bin_diff(diff):
    diff = round(diff, 2)
    if diff == 0:
        return 'None'
    elif diff >= 0.1 and diff <= 0.5:
        return 'Low'
    elif diff > 0.5 and diff <= 1.0:
        return 'Medium'
    elif diff > 1.0 and diff <= 1.5:
        return 'High'

## Function: Transform the data and create necessary additional columns
def transform_data(df):
    df['zom_per'] = df.apply(lambda x: round((x['zom_rtg_count']*100)/(x['zom_rtg_count']+x['yelp_rtg_count']),2), axis=1)
    df['yelp_per'] = df.apply(lambda x: round((x['yelp_rtg_count']*100)/(x['zom_rtg_count']+x['yelp_rtg_count']),2), axis=1)
    df['diff_signed'] = df['zom_rating'] - df['yelp_rating']
    df['diff'] = df.apply(lambda x: (x['diff_signed'] * -1) if x['diff_signed'] < 0 else x['diff_signed'], axis=1)
    df['bin'] = df.apply(lambda x: bin_diff(x['diff']), axis=1)
    return df

## Function: Plot shows the distribution of restaurants along the polarized rating bins
def diff_bin_plot_1(df):
    x = df['bin'].value_counts()
    x = x.reindex(index = ['None','Low','Medium','High'])
    x = x.rename(index={'Low':'Low\n(0.1 to 0.5)', 'Medium':'Medium\n(0.6 to 1.0)', 'High':'High\n(1.1 to 1.5)'})
    plt.figure(figsize=(8,7))
    colours = ['grey' if (i > min(x.values)) else 'red' for i in x.values ]
    sb.set(font_scale = 1.2)
    ax = sb.barplot(x.index, x.values, palette=colours, alpha=0.9)
    plt.title("Polarization of Restaurant Ratings", fontsize=24, fontweight='bold')
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=0, fontsize=14)
    plt.ylabel('# of Restaurants', fontsize=14, fontweight='bold')
    plt.xlabel('Differences in Ratings Across Zomato & Yelp', fontsize=14, fontweight='bold')

    #adding the text labels
    rects = ax.patches
    labels = x.values
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')
    plt.show()

## Function: Gives marker background color for the map point of the restaurant based on its rating bin
def get_marker_colour(rating_bin):
    if rating_bin == 'None':
        return 'white'
    elif rating_bin == 'Low':
        return 'lightgray'
    elif rating_bin == 'Medium':
        return 'gray'
    elif rating_bin == 'High':
        return 'red'

## Function: Displays the map of the restaurants having a 'High' rating difference
def folium_plot_2(df):
    df = df[df['bin']=='High']
    m = folium.Map(location=[41.878113, -87.629799], tiles='Stamen Terrain', zoom_start=5,control_scale=True)
    
    for i in range(len(df)):
        folium.Marker([df.iloc[i]['lat'], df.iloc[i]['long']], \
                popup="<strong>"+df.iloc[i]['name']+" ("+str(round(df.iloc[i]['diff'],2))+")""</strong>", \
                icon=folium.Icon(color=get_marker_colour(df.iloc[i]['bin']), icon_color='black', icon='glyphicon glyphicon-cutlery', angle=0)) \
                .add_to(m)        
    display(m)

def draw_cusine(df, site, heading):
    x=df.category.value_counts()
    x=x.sort_values(ascending=False)
    x=x.iloc[0:6]
    
    plt.figure(figsize=(8,7))
    colours = ['grey' if (i < max(x.values)) else 'red' for i in x.values ]
    sb.set(font_scale = 1.2)
    ax = sb.barplot(x.index, x.values, palette=colours, alpha=0.9)
    ax.tick_params(labelsize=12)
    plt.title(heading, fontsize=20, fontweight='bold')
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=45, fontsize=12)
    plt.ylabel('# of Restaurants', fontsize=14, fontweight='bold')
    plt.xlabel('Cuisines', fontsize=14, fontweight='bold')

    #adding the text labels
    rects = ax.patches
    labels = x.values
    for rect, label in zip(rects, labels):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2, height, label, ha='center', va='bottom')
    plt.show()

## Function: Plots of top 6 cuisines in each rating difference category 
def cuisine_plot_3(df):
    for level in ['Low', 'Medium', 'High']:
        zom_id_high = df[df['bin']==level]['zom_id'].tolist()
        zomato = get_data('select * from restaurant')
        zomato = zomato[zomato['restaurant_id'].isin(zom_id_high)]
        zom_cat = ', '.join(zomato['cuisines']).split(', ') # cuisine list
        zom_df = pd.DataFrame(zom_cat, columns=['category'])
        draw_cusine(zom_df, 'zomato', 'Zomato Cuisines of \''+level+'\' Polarized Restaurants')  
        
        yelp_id_high = df[df['bin']==level]['yelp_id'].tolist()
        yelp = get_data('select * from business')
        yelp = yelp[yelp['business_id'].isin(yelp_id_high)]
        yelp_cat = ', '.join(yelp['categories']).split(', ')
        # Ignore 'Restaurants' and 'Food' since they are common in most restaurants
        yelp_cat = [ele for ele in yelp_cat if ele not in {'Restaurants','Food'}] 
        yelp_df = pd.DataFrame(yelp_cat, columns=['category'])
        draw_cusine(yelp_df, 'yelp', 'Yelp Categories of \''+level+'\' Polarized Restaurants')

## Function: 2nd set of Visualizations
def run_visualization_2():
    df = get_data() # df is the dataframe with common 391 retaurants between zomato and yelp
    df = transform_data(df)
    diff_bin_plot_1(df)
    folium_plot_2(df)
    cuisine_plot_3(df)

## Topic 4: Data Analysis - Analysing Yelp's Review Dataset using Sentiment Analysis using Wordclouds 

In [ ]:
#basics
import numpy as np
import pandas as pd

#misc
import gc
import time
import warnings

#viz
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.gridspec as gridspec 
import matplotlib.gridspec as gridspec 

# graph viz
import plotly.offline as pyo
from plotly.graph_objs import *
import plotly.graph_objs as go

from collections import Counter 
from wordcloud import WordCloud, STOPWORDS # wordcloud in python

import re 
import string
import nltk # preprocessing text
from textblob import TextBlob

def sentiment(x):
    sentiment = TextBlob(x)
    return sentiment.sentiment.polarity

# function for pre-processing the text of reviews: this function remove punctuation, stopwords and returns the list of words
def preprocess(x):
    x = re.sub('[^a-z\s]', '', x.lower())                  
    x = [w for w in x.split() if w not in set(stopwords)]  
    return ' '.join(x)

## Function: 3rd set of Visualizations
def run_visualization_3():
    review=pd.read_csv("review match.csv")
    nltk.download("stopwords")

    #removing stop words
    i = nltk.corpus.stopwords.words('english')
    # punctuations to remove
    j = list(string.punctuation)
    # finally let's combine all of these
    stopwords = set(i).union(j).union(('thiswas','wasbad','thisis','wasgood','isbad','isgood','theres','there'))4

    review['text_processed'] = review['text'].apply(preprocess)

    #Top reviewed words
    wordcloud = WordCloud(width=1600, height=800, random_state=1, max_words=500, background_color='white',)
    wordcloud.generate(str(set(review['text_processed'])))
    # declare our figure 
    plt.figure(figsize=(20,10))
    plt.title("Top Reviewed words", fontsize=40,color='Red')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.tight_layout(pad=10)
    plt.show()

    def sentiment(x):
        sentiment = TextBlob(x)
        return sentiment.sentiment.polarity

    review['text_sentiment'] = review['text_processed'].apply(sentiment)

    review['sentiment'][review['text_sentiment'] > 0] = 'positive'
    review['sentiment'][review['text_sentiment'] < 0] = 'negative'
    review['sentiment'][review['text_sentiment'] == 0] = 'neutral'

    #Review Sentiments count
    plt.figure(figsize=(6,6))
    ax = sns.countplot(review['sentiment'])
    plt.title('Review Sentiments');

    review_posr = pd.DataFrame(review['text_processed'][ review['sentiment'] == 'positive'])
    review_negr = pd.DataFrame(review['text_processed'][ review['sentiment'] == 'negative'])
    review_neutr = pd.DataFrame(review['text_processed'][ review['sentiment'] == 'neutral'])

    #Positive Sentiment cloud
    wordcloud = WordCloud(width=1600, height=800, random_state=1, max_words=500, background_color='white',)
    wordcloud.generate(str(set(review_posr['text_processed'])))
    # declare our figure 
    plt.figure(figsize=(20,10))
    plt.title("Positive Sentiment", fontsize=40,color='Red')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.tight_layout(pad=10)
    plt.show()

    #Negative sentiment cloud
    wordcloud = WordCloud(width=1600, height=800, random_state=1, max_words=500, background_color='white',)
    wordcloud.generate(str(set(review_negr['text_processed'])))
    # declare our figure 
    plt.figure(figsize=(20,10))
    plt.title("Negative Sentiment", fontsize=40,color='Red')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.tight_layout(pad=10)
    plt.show()

    wordcloud = WordCloud(width=1600, height=800, random_state=1, max_words=500, background_color='white',)
    wordcloud.generate(str(set(review_neutr['text_processed'])))
    # declare our figure 
    plt.figure(figsize=(20,10))
    plt.title("Neutral Sentiment", fontsize=40,color='Red')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.tight_layout(pad=10)
    plt.show()



## Main Function (Start Point)

In [6]:
#
#
# Main Function:
#
#
def main():
    run_extraction_storage()
    run_visualization_1()
    run_visualization_2()
    run_visualization_3()
    pass
    
if __name__ == "__main__":
    main()